In [11]:
######## 10.1 FashinMNIST 다운로드: 데이터 불러오고 서브셋 만듦

from itertools import chain # 여러 리스트를 하나의 리스트로 연결
from collections import defaultdict # 기본값이 리스트인 딕셔너리 생성
from torch.utils.data import Subset
from torchvision import datasets


def subset_sampler(dataset, classes, max_len):
    target_idx = defaultdict(list) # 각 클래스의 인덱스를 저장하는 dict
    # dataset.train_labels=각 이미지에 해당하는 클래스(label)(0~9)의 정수 배열
    for idx, label in enumerate(dataset.train_labels):
        target_idx[int(label)].append(idx)

    # 각 클래스별로 max_len만큼의 인덱스를 추출하여 하나의 리스트로 만듦
    indices = list(
        chain.from_iterable(
            [target_idx[idx][:max_len] for idx in range(len(classes))]
        )
    )
    return Subset(dataset, indices)


train_dataset = datasets.FashionMNIST(root="../datasets", download=True, train=True)
test_dataset  = datasets.FashionMNIST(root="../datasets", download=True, train=False)

# train dataset에 포함된 클래스
classes = train_dataset.classes
# train dataset의 클래스와 클래스id가 매핑된 값
class_to_idx = train_dataset.class_to_idx

print(classes)
print(class_to_idx)

# 각 클래스별로 최대 1000개의 샘플을 포함하는 훈련 데이터 서브셋 만듦
subset_train_dataset = subset_sampler(
    dataset = train_dataset, classes = train_dataset.classes, max_len = 1000
)
# 각 클래스별로 최대 100개의 샘플을 포함하는 테스트 데이터 서브셋 만듦
subset_test_dataset = subset_sampler(
    dataset = test_dataset, classes = test_dataset.classes, max_len = 100
)

print(f"Training Data Size : {len(subset_train_dataset)}") # max_len x len(classes) = 1000x10
print(f"Testing Data Size : {len(subset_test_dataset)}")
print(train_dataset[0], subset_train_dataset[0]) #(PIL이미지, class label)

['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
{'T-shirt/top': 0, 'Trouser': 1, 'Pullover': 2, 'Dress': 3, 'Coat': 4, 'Sandal': 5, 'Shirt': 6, 'Sneaker': 7, 'Bag': 8, 'Ankle boot': 9}
Training Data Size : 10000
Testing Data Size : 1000
(<PIL.Image.Image image mode=L size=28x28 at 0x2AF44BD50>, 9) (<PIL.Image.Image image mode=L size=28x28 at 0x2AF44BDD0>, 0)


/opt/homebrew/lib/python3.11/site-packages/torchvision/datasets/mnist.py:65: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")


In [12]:
######## 10.2 이미지 전처리
import torch
from torchvision import transforms
from transformers import AutoImageProcessor

# 사전학습된 ViT 모델 활용해 전처리 진행
image_processor = AutoImageProcessor.from_pretrained(
    pretrained_model_name_or_path = "google/vit-base-patch16-224-in21k"
)

transform = transforms.Compose(
    [
        # 모델 학습을 위해 PIL.image > Tensor
        transforms.ToTensor(),
        # 크기 조정
        transforms.Resize(
            size=(
                image_processor.size["height"],
                image_processor.size["width"]
            )
        ),
        # 단일 채널을 복제해 다중 채널 이미지로 변환
        transforms.Lambda(
            # FashinMNIST의 x는 흑백이미지 [1,H,W]
            # 텐서 3개로 복사하여 0번차원 기준으로 cat(연결)
            #결과: [3,H,W]
            lambda x: torch.cat([x, x, x], 0)
        ),
        # 정규화
        transforms.Normalize(
            mean = image_processor.image_mean,
            std  = image_processor.image_std
        )
    ]
)

print(f"size : {image_processor.size}")
print(f"mean : {image_processor.image_mean}")
print(f"std : {image_processor.image_std}")

size : {'height': 224, 'width': 224}
mean : [0.5, 0.5, 0.5]
std : [0.5, 0.5, 0.5]


In [13]:
######## 10.3 ViT 모델 구조에 맞는 형태로 변환해데이터로더 적용
from torch.utils.data import DataLoader

# ViT모델은 Tensor 형식이 아닌 딕셔너리 형식의 데이터를 입력으로 받음
# ViT 모델의 입력: {"pixel_values":pixel_values, "labels":labels}

def collator(data, transform):
    images, labels = zip(*data)
    # pixel_values = (배치크기, 채널수, 이미지높이, 이미지너비)
    pixel_values = torch.stack([transform(image) for image in images])
    # labels = [클래스 색인값]
    labels       = torch.tensor([label for label in labels])
    return {"pixel_values": pixel_values, "labels": labels}


train_dataloader = DataLoader(
    subset_train_dataset, # (PIL이미지, label)
    batch_size = 32,
    shuffle    = True,
    collate_fn = lambda x: collator(x, transform),
    drop_last  = True
)
valid_dataloader = DataLoader(
    subset_test_dataset,
    batch_size = 4,
    shuffle    = True,
    collate_fn = lambda x: collator(x, transform),
    drop_last  = True
)

batch = next(iter(train_dataloader))
# batch = {"pixel_values":(배치크기, 채널수, 이미지높이, 이미지너비), "labels":배치 크기의 클래스 색인 list}
for key, value in batch.items():
    print(f"{key} : {value.shape}")

pixel_values : torch.Size([32, 3, 224, 224])
labels : torch.Size([32])


In [14]:
######## 10.4 사전 학습된 ViT 모델
from transformers import ViTForImageClassification


model = ViTForImageClassification.from_pretrained(
    # 사전 학습된 모델은 앞의 이미지 프로세서 클래스에서 사용한 모델과 동일
    pretrained_model_name_or_path = "google/vit-base-patch16-224-in21k",
    # 아래의 3개 매개변수를 통해 현재 데이터세트에 적합한 구조로 모델을 미세조정
    num_labels                    = len(classes),
    id2label                      = {idx: label for label, idx in class_to_idx.items()},
    label2id                      = class_to_idx,
    ignore_mismatched_sizes       = True
)

print(model.classifier)

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Linear(in_features=768, out_features=10, bias=True)


In [15]:
######## 10.5 패치 임베딩 확인
print(model.vit.embeddings)

batch = next(iter(train_dataloader))
# 기존 이미지 (배치사이즈, 채널수, 이미지높이, 이미지너비)
print("image shape :", batch["pixel_values"].shape)

# 임베딩 후 (배치크기, 패치수(14x14), 임베딩길이)
print("patch embeddings shape :",
    model.vit.embeddings.patch_embeddings(batch["pixel_values"]).shape
)
print("[CLS] + patch embeddings shape :",
    model.vit.embeddings(batch["pixel_values"]).shape
)

ViTEmbeddings(
  (patch_embeddings): ViTPatchEmbeddings(
    (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  )
  (dropout): Dropout(p=0.0, inplace=False)
)
image shape : torch.Size([32, 3, 224, 224])
patch embeddings shape : torch.Size([32, 196, 768])
[CLS] + patch embeddings shape : torch.Size([32, 197, 768])


In [16]:
# !pip install transformers==4.30.2 accelerate==0.21.0

In [17]:
######## 10.6 하이퍼파라미터 설정
from transformers import TrainingArguments

# TrainingArguments(학습매개변수클래스): 모델 학습에 필요한 다양한 인자들을 저장하고 관리
args = TrainingArguments(
    output_dir                  = "../models/ViT-FashionMNIST", # 체크포인트 저장경로
    save_strategy               = "epoch", # 체크포인트 저장간격
    evaluation_strategy         = "epoch", # 체크포인트 평가간격
    learning_rate               = 1e-5,
    per_device_train_batch_size = 16,      # 학습 배치 크기
    per_device_eval_batch_size  = 16,      # 평가 배치 크기
    num_train_epochs            = 3,       # 학습 반복 수
    weight_decay                = 0.001,
    load_best_model_at_end      = True,
    metric_for_best_model       = "f1",     # 최상의 모델 선정 기준 평가 방식
    logging_dir                 = "logs",   # 로그 저장 폴더
    logging_steps               = 125,      # 로그 출력 간격
    remove_unused_columns       = False,
    seed                        = 7
)

In [18]:
######## 10.7 매크로 평균 F1 점수
import evaluate
import numpy as np


def compute_metrics(eval_pred):
    metric = evaluate.load("f1")
    # predictions: 모든 클래스에 대한 예측값, labels: 실제값
    predictions, labels = eval_pred 
    # 가장 확률이 높은 값을 저장
    predictions = np.argmax(predictions, axis=1)
    # f1 점수 계산
    macro_f1    = metric.compute(
        # 매크로 평균 F1 점수 방식 사용하므로 average매개변수에 macro 입력
        predictions = predictions, references = labels, average = "macro"
    )
    return macro_f1

In [1]:
######## 10.8 ViT 모델 학습 (Colab에서 실행)
import torch
from transformers import ViTForImageClassification
from transformers import Trainer

device = torch.device('mps')


def model_init(classes, class_to_idx):
    model = ViTForImageClassification.from_pretrained(
        pretrained_model_name_or_path = "google/vit-base-patch16-224-in21k",
        num_labels                    = len(classes),
        id2label                      = {idx: label for label, idx in class_to_idx.items()},
        label2id                      = class_to_idx,
    )
    return model


# Trainer 클래스를 통해 ViT 모델 학습 수행
trainer = Trainer(
    # model_init      = model_init,
    model_init      = lambda x: model_init(classes, class_to_idx),
    args            = args,
    train_dataset   = subset_train_dataset,
    eval_dataset    = subset_test_dataset,
    data_collator   = lambda x: collator(x, transform),
    compute_metrics = compute_metrics,
    tokenizer       = image_processor,
)
trainer.train()

![](../screenshot/vit-train.png)

최종학습의 손실값은 0.4049이며 F1 점수가 0.9232로 학습이 우수하게 진행된 것을 확인할 수 있다.

In [2]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


outputs = trainer.predict(subset_test_dataset)
print(outputs)

y_true = outputs.label_ids
y_pred = outputs.predictions.argmax(1)

labels  = list(classes)
# confusion_matrix 활용해 성능 평가
matrix  = confusion_matrix(y_true, y_pred)
display = ConfusionMatrixDisplay(confusion_matrix=matrix, display_labels=labels)
_, ax   = plt.subplots(figsize=(10, 10))
display.plot(xticks_rotation=45, ax=ax)
plt.show()

# 혼동 행렬을 통해 오분류된 클래스에 대한 문제점을 파악하고 모델을 개선해 정확도를 향상시킬 수 있음
# 주요한 모델 개선 방법: 하이퍼파라미터 조정, 모델그조 변경, 전처리 과정 개선, 데이터 증강 등

metrics={'test_loss': 0.43413683772087097, 'test_f1': 0.9232480514680315, 'test_runtime': 13.459, 'test_samples_per_second': 74.3, 'test_steps_per_second': 4.681}



![](../screenshot/vit-test2.png)